<a href="https://colab.research.google.com/github/jeffersondemota/projeto_/blob/main/compfrete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================================
# 🧭 Sistema de Comparação de Custos Logísticos
# Frota Própria x Agregada x CrossDocking
# Executável direto no Google Colab via Streamlit + ngrok
# =========================================================

# === 1️⃣ Criação dos arquivos app.py e run_in_colab.py ===

app_code = r'''
import os
import io
import math
import json
import time
import base64
import requests
import pandas as pd
import numpy as np
import streamlit as st
from datetime import datetime, timedelta
from PIL import Image # Import Pillow library to handle images

# --- Funções auxiliares (resumidas para clareza) ---
def safe_float(x, default=0.0):
    try: return float(str(x).replace(",", "."))
    except: return default

def to_hours(x):
    if pd.isna(x): return 0
    s=str(x)
    if ":" in s:
        h,m,*_ = s.split(":")
        return float(h)+float(m)/60
    try:
        # Assume numerical input is in minutes and convert to hours
        return float(s) / 60
    except:
        return 0

# Modified OSRM function to return polyline
def osrm_distance_duration_polyline(coords):
    if len(coords)<2: return 0,0,None
    base = "https://router.project-osrm.org/route/v1/driving/"
    path = ";".join([f"{c[0]:.6f},{c[1]:.6f}" for c in coords])
    url = f"{base}{path}?overview=full&geometries=polyline6"
    try:
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            data = r.json()
            if data.get("routes"):
                route = data["routes"][0]
                dist_km = route["distance"] / 1000.0
                dur_h = route["duration"] / 3600.0
                poly6 = route.get("geometry")  # polyline for the toll API
                return dist_km, dur_h, poly6
    except Exception:
        pass
    return 0.0, 0.0, None


def vehicle_class(p):
    if p<=600: return "UTILITARIO_600"
    elif p<=1600: return "VAN_1600"
    elif p<=3000: return "TRES_QUARTOS_3000"
    elif p<=5000: return "SEMI_TOCO_5000"
    elif p<=14000: return "TRUCK"
    else: return "CARRETA"

def map_col(v):
    return {
        "UTILITARIO_600":"FRETE UTILITÁRIO 600Kg",
        "VAN_1600":"FRETE VAN 1600Kg",
        "TRES_QUARTOS_3000":"FRETE 3/4 3000Kg",
        "SEMI_TOCO_5000":"FRETE SEMI - TOCO 5000Kg",
        "TRUCK":"FRETE TRUCK",
        "CARRETA":"FRETE CARRETA"
    }.get(v)

def interpolate(df, km_col, val_col, km):
    if val_col not in df.columns: return np.nan
    df=df.sort_values(km_col)
    x=df[km_col].astype(float).values; y=df[val_col].astype(float).values
    if km<=x.min(): return y[0]
    if km>=x.max(): return y[-1]
    i=np.searchsorted(x,km)
    x0,x1=x[i-1],x[i]; y0,y1=y[i-1],y[i]
    return y0+(y1-y0)*(km-x0)/(x1-0)

# New function to map vehicle class to axles
def eixo_por_classe(vclass):
    # adjust according to your real fleet
    return {
        "UTILITARIO_600": 2,
        "VAN_1600": 2,
        "TRES_QUARTOS_3000": 2,
        "SEMI_TOCO_5000": 3,
        "TRUCK": 3,
        "CARRETA": 5,   # can be 5–9; adjust according to trailer
    }.get(vclass, 2)

# New function to call TollGuru API (provided by user)
def toll_cost_for_route_tollguru(polyline6, vehicle_axles, api_key=None):
    """
    Calcula pedágio via TollGuru a partir de uma polilinha (polyline6) e eixos do veículo.
    Retorna custo total em BRL ou 0.0 se falhar.
    """
    if not api_key or not polyline6:
        return 0.0
    url = "https://dev.tollguru.com/v1/calc/route"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    payload = {
        "source": "OSRM",
        "polyline": polyline6,
        "vehicleType": "truck",        # ou "car" se aplicável
        "vehicle": {
            "axles": vehicle_axles
        },
        "country": "BR", # Changed to fixed "BR"
        "currency": "BRL"
    }
    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=30)
        if resp.status_code == 200:
            data = resp.json()
            # verifique o caminho do total (varia por versão); exemplo comum:
            total = (
                data.get("route", {})
                    .get("costs", {})
                    .get("tag", {})
                    .get("currency", "BRL")
            )
            # fallback direto:
            total_val = data.get("route", {}).get("costs", {}).get("tag", {}).get("amount")
            if total_val is None:
                # outro formato possível
                total_val = data.get("route", {}).get("costs", {}).get("cash", {}).get("amount", 0.0)
            return float(total_val or 0.0)
        else:
             st.warning(f"TollGuru API returned status code: {resp.status_code} - {resp.text}")
             return 0.0
    except Exception as e:
        st.warning(f"Error calling TollGuru API: {e}")
        pass
    return 0.0


st.set_page_config(page_title="Comparador de Custos", layout="wide")
# Add the image logo
try:
    image = Image.open('/content/drive/MyDrive/Colab Notebooks/logo-branca-qfpsii0s3y2kjypwqu57rvsp16k4hj6dc0rz3dj1ia.png')
    st.image(image)
except FileNotFoundError:
    st.warning("Logo image not found. Make sure 'logo-branca-qfpsii0s3y2kjypwqu57rvsp16k4hj6dc0rz3dj1ia.png' is uploaded to the /content/ directory.") # Added specific directory instruction

st.title("🚛 Comparador de Custos de transporte ARCHOTE: Frota Própria x Agregada x CrossDocking")

with st.sidebar:
    st.header("⚙️ Parâmetros")
    total_fixo_per_day = 133.08 # Modified variable name with new value
    st.write("**Custo fixo total/dia:**",round(total_fixo_per_day,2)) # Used new variable name
    km_l=st.number_input("Consumo (km/L)",5.0)
    preco_comb=st.number_input("Preço Combustível",5.99)

    # Added selectbox for toll method
    pedagio_provider = st.selectbox("Pedágio - Método", ["manual (taxa R$/km)", "TollGuru (API real)"])
    tollguru_key = None
    per_km_toll = 0.0
    if "TollGuru" in pedagio_provider:
        tollguru_key = st.text_input("TollGuru API Key", type="password")
    else:
        per_km_toll = st.number_input("Taxa de Pedágio (R$/km)", min_value=0.0, value=0.45, step=0.05, format="%.2f")


    j_ini,j_fim=6,16
    # Removed custo_h_extra input as it will be calculated
    base_lat=st.number_input("Base Lat",-23.603)
    base_lon=st.number_input("Base Lon",-46.919)

    st.subheader("Uploads")
    cli=st.file_uploader("RelatorioCadastroCliente")
    rota=st.file_uploader("rota")
    tab=st.file_uploader("TabelaFreteCustosFrotaAgregada")
    cross=st.file_uploader("CustosCrossDocking")
    run=st.button("🚀 Executar")

def read_df(f):
    if f is None: return None
    n=f.name.lower()
    if n.endswith(".csv"): return pd.read_csv(f)
    return pd.read_excel(f)

if run:
    df_cli,df_rota,df_tab,df_cross=[read_df(x) for x in [cli,rota,tab,cross]]
    if any(df is None for df in [df_cli,df_rota,df_tab,df_cross]):
        st.error("Faltam planilhas!"); st.stop()
    if "TollGuru" in pedagio_provider and not tollguru_key: # Check if API key is provided when TollGuru is selected
        st.error("TollGuru API Key é necessária para calcular pedágios."); st.stop()


    df_cli.rename(columns=lambda x:x.strip(),inplace=True)
    df_rota.rename(columns=lambda x:x.strip().upper(),inplace=True)
    df_tab.rename(columns=lambda x:x.strip(),inplace=True)
    df_cross.rename(columns=lambda x:x.strip().upper(),inplace=True)
    df_cli_m=df_cli[["CódigoCliente","Latitude","Longitude","TempoMédioEntrega"]]
    df_cli_m.columns=["IDCLI","LAT","LON","TEMPO"]
    df=df_rota.merge(df_cli_m,on="IDCLI",how="left")

    resultados=[]
    detalhe_frota_propria = [] # New list for detailed own fleet cost
    detalhe_tempo_jornada = [] # New list for detailed journey time
    base=(base_lon,base_lat)
    custo_motorista_fixo = 164.25 # Fixed driver cost
    custo_indireto_fixo = 232.86 # Fixed indirect cost
    custo_ajudante_fixo = 95.72 # Fixed helper cost

    for vid,g in df.groupby("IDVEICULO"):
        coords=[base]+[(float(x.LON),float(x.LAT)) for _,x in g.iterrows()]+[base]
        km,hr, polyline = osrm_distance_duration_polyline(coords) # Updated to receive polyline
        tempo=g["TEMPO"].apply(to_hours).sum()
        total_h=hr+tempo
        extra=max(0,total_h-(j_fim-j_ini))
        # Calculate Hora trabalhada and Custo Hora Extra based on MDO
        custo_motorista = custo_motorista_fixo
        custo_indireto = custo_indireto_fixo
        custo_ajudante = custo_ajudante_fixo # Added helper cost

        # Check if number of unique clients is greater than 1 and double helper cost
        num_unique_clients = g["IDCLI"].nunique()
        if num_unique_clients > 1:
            custo_ajudante = custo_ajudante * 2

        total_custo_mdo = custo_motorista + custo_ajudante # Calculate total MDO cost
        hora_trabalhada_rate = total_custo_mdo / 9 if total_custo_mdo > 0 else 0 # Calculate Hora trabalhada rate, avoid division by zero
        custo_extra = extra * hora_trabalhada_rate * 1.40 # Calculate Custo Hora Extra with 40% premium

        comb=(km/km_l)*preco_comb
        # Calculate toll cost based on selected method
        peso=g["PESO"].sum()
        vcl=vehicle_class(peso)
        axles = eixo_por_classe(vcl) # Get axles based on vehicle class

        if "TollGuru" in pedagio_provider:
            ped = toll_cost_for_route_tollguru(polyline, axles, tollguru_key) # Calculate toll cost using API
        else:
            ped = km * per_km_toll # Calculate toll cost using manual rate

        # aqui é o calculo de pedágio
        ped = ped * 2 # Multiply toll cost by 2 as requested
        ped = ped + (axles * 1.0) # Add 1 real per axle

        frota_propria=total_fixo_per_day+comb+ped+custo_extra+custo_motorista+custo_indireto+custo_ajudante # Included all fixed costs
        col=map_col(vcl)
        km_ida=km/2
        frete=interpolate(df_tab,"KM IDA",col,km_ida) if col else np.nan
        cross_v=peso * 0.38
        resultados.append([vid,round(km,1),peso,frota_propria,frete,cross_v,vcl, num_unique_clients])

        # Append detailed own fleet cost for this vehicle, including all fixed costs and total MDO
        detalhe_frota_propria.append([vid, total_fixo_per_day, comb, ped, custo_extra, custo_motorista, custo_indireto, custo_ajudante, total_custo_mdo, frota_propria]) # Added total MDO cost to detailed list

        # Append detailed journey time for this vehicle
        detalhe_tempo_jornada.append([vid, tempo, hr, total_h, extra]) # Added extra hours to detailed journey time list

    out=pd.DataFrame(resultados,columns=["IDVEICULO","KM_TOTAL","PESO","CUSTO_FROTA_PRORIA","CUSTO_FROTA_AGREGADA","CUSTO_CROSS","VEICULO", "NUM_UNIQUE_CLIENTS"])
    out["MODAL_MAIS_BARATO"]=out[["CUSTO_FROTA_PRORIA","CUSTO_FROTA_AGREGADA","CUSTO_CROSS"]].idxmin(axis=1)
    st.dataframe(out)
    # st.bar_chart(out.melt(id_vars="IDVEICULO",value_vars=["CUSTO_FROTA_PRORIA","CUSTO_FROTA_AGREGADA","CUSTO_CROSS"])) # Removed the bar chart

    # Display the second table for detailed own fleet cost
    st.subheader("Detalhe do Custo da Frota Própria") # Subheader for the new table
    df_detalhe = pd.DataFrame(detalhe_frota_propria, columns=["IDVEICULO", "Custo Fixo Diário", "Custo Combustível", "Custo Pedágio", "Custo Hora Extra", "Custo Motorista", "Custo Indireto", "Custo Ajudante", "Total Custo MDO", "Total Frota Própria"]) # New DataFrame, added all fixed cost columns and Total Custo MDO
    st.dataframe(df_detalhe) # Display the new DataFrame

    # Display the third table for detailed journey time
    st.subheader("Detalhe do Tempo de Jornada") # Subheader for the third table
    df_tempo = pd.DataFrame(detalhe_tempo_jornada, columns=["IDVEICULO", "Tempo Médio Entrega (horas)", "Tempo de Trajeto Estimado (horas)", "Total Horas Jornada", "Total de Hora Extra"]) # New DataFrame, added "Total de Hora Extra" column
    st.dataframe(df_tempo) # Display the new DataFrame

    # Export all dataframes to different sheets in one Excel file
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        out.to_excel(writer, sheet_name='Comparativo Custos', index=False)
        df_detalhe.to_excel(writer, sheet_name='Detalhe Frota Propria', index=False)
        df_tempo.to_excel(writer, sheet_name='Detalhe Tempo Jornada', index=False)

    output.seek(0)

    st.download_button(
        label="Baixar Excel Completo",
        data=output,
        file_name="Comparativo_Custos_Transporte_Completo.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )

else:
    st.info("Carregue planilhas e clique em Executar")
'''

launcher_code = r'''
import os, subprocess, time, sys
from pyngrok import ngrok
from PIL import Image # Import Pillow for image handling

def ensure_installed():
    import pkgutil
    pkgs=["streamlit","pyngrok","pandas","numpy","requests","xlsxwriter","Pillow"] # Added Pillow
    need=[p for p in pkgs if pkgutil.find_loader(p) is None]
    if need: subprocess.check_call([sys.executable,"-m","pip","install",*need])

def run():
    port=8501
    token=os.environ.get("NGROK_AUTH_TOKEN","")
    if token: ngrok.set_auth_token(token)

    # Disconnect any existing ngrok tunnels
    try:
        tunnels = ngrok.get_tunnels()
        if tunnels:
            print("Disconnecting existing tunnels...")
            for tunnel in tunnels:
                print(f"  Disconnecting {tunnel.public_url}")
                ngrok.disconnect(tunnel.public_url)
    except Exception as e:
        print(f"Error disconnecting tunnels: {e}")

    url=ngrok.connect(port).public_url
    print("🌐 Public URL:",url)
    p=subprocess.Popen(["streamlit","run","app.py"])
    try:
        while True: time.sleep(5)
    except KeyboardInterrupt: p.terminate()

if __name__=="__main__":
    ensure_installed(); run()
'''

with open("app.py","w",encoding="utf-8") as f: f.write(app_code)
with open("run_in_colab.py","w",encoding="utf-8") as f: f.write(launcher_code)

print("✅ Arquivos criados: app.py e run_in_colab.py")

# === 2️⃣ Instala dependências e executa Streamlit com ngrok ===
!pip install streamlit pyngrok pandas numpy requests xlsxwriter Pillow -q # Added Pillow

import os
os.environ["NGROK_AUTH_TOKEN"] = "33vthvkGJREkZID7YROACMe1BMT_KWnW3rpre6rUhGJQV282"  # <- Substitua aqui pelo seu token ngrok

!python run_in_colab.py

✅ Arquivos criados: app.py e run_in_colab.py
/content/run_in_colab.py:9: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  need=[p for p in pkgs if pkgutil.find_loader(p) is None]
🌐 Public URL: https://sammy-cybernetic-denice.ngrok-free.dev



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.228.185:8501

